## Go to the colab notebook for details


In [ ]:
pip install tensorflow-io

In [ ]:
pip install python-gdcm

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
t_image_fnames = []
path = "/kaggle/input/siim-covid19-detection/train/"
import os
len(os.listdir(path))
for root, dirs, filenames in os.walk(path):
    for fname in filenames:
        t_image_fnames.append(os.path.join(root,fname))

train_image_level = pd.read_csv("/kaggle/input/siim-covid19-detection/train_image_level.csv")
train_study_level = pd.read_csv("/kaggle/input/siim-covid19-detection/train_study_level.csv")    
len(t_image_fnames)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
if len(train_image_level.id) == len(t_image_fnames):
    print("length is almost the same")
    
else:
    print("you fucked up")
    
train_image_level.head()

In [ ]:
train_image_level.isnull().sum()

In [ ]:
# verifying the directory stucture.
for i in train_image_level["StudyInstanceUID"]:
    if i == "0135d267f462":
        print("its here")
        break
train_image_level.id = train_image_level.id.str.split("_",expand = True,n=0)

# The number of cases with more than one images

train_image_level.StudyInstanceUID.duplicated().sum()        

## Creating our own validation set

Now gotta create the datasets and create our own validation set


In [ ]:
NUM_IMAGES = 200 #@param {type:"slider", min:100, max:6334, step: 10 }

In [ ]:
train_image_level.head()

In [ ]:
### Setup X & y variables

X = t_image_fnames
y = train_image_level.label
bboxes = train_image_level.boxes

In [ ]:
y[1]

## Lets see what improting an image looks like:



In [ ]:
import matplotlib.pyplot as plt

import pydicom
%matplotlib inline
plt.figure(figsize = (10,8))
image = pydicom.dcmread(X[21])
plt.imshow(image.pixel_array,cmap=plt.cm.bone)


## Function to show 25 Images


In [ ]:
import matplotlib.pyplot as plt
import pydicom

def show_25_images(images):
    """
    Displays a plot of 25 images and their labes for training images
    """
    
    # setup the figure
    plt.figure(figsize = (10,10))
    
    # loop through 25 files to display 25 images
    for i in range(25):
        # Create subplots ( 5 rows , 5 columns)
        ax = plt.subplot(5,5,i+1)
        # display an image
        image = pydicom.dcmread(images[i])
        plt.imshow(image.pixel_array,cmap = plt.cm.bone)
        plt.axis("off")
        
        

In [ ]:
show_25_images(X)

In [ ]:
len(X), len(y), len(bboxes)

## Preprocessing Images (Turning images into tensors)

Write a function to preprocess images
* 1. Take an image filepath as input
* 2. Use Tensorflow to read the file and save it to a variable, `image`
* 3. Turn our image (dicom) into Tensors
* 4. Resize the image to a shape of (416,416) # Hyper parameter to experiment with
* 5. Return the modified image

In [ ]:
import matplotlib.pyplot as plt
import pydicom 
%matplotlib inline
plt.figure(figsize = (10,8))
image = pydicom.dcmread(X[2])
plt.imshow(image.pixel_array,cmap=plt.cm.bone)

In [ ]:
import ast
boxes = ast.literal_eval(train_image_level.loc[3,'boxes'])
boxes

In [ ]:
img = pydicom.dcmread(X[3])
img = img.pixel_array
img

In [ ]:
import matplotlib
fig,ax = plt.subplots(figsize=(10,8))

for box in boxes:
    p = matplotlib.patches.Rectangle((box['x'], box['y']),box['width'], box['height'],
                                     ec='r' , fc = 'none',lw = 2.)
    ax.add_patch(p)
    
ax.imshow(img, cmap= plt.cm.bone)
plt.show()
                                     

In [ ]:
fig, axs = plt.subplots(3,3,figsize = (20,16))
fig.subplots_adjust(hspace = .1 , wspace = .1)
axs = axs.ravel()

for row in range(9):
    study = train_image_level.loc[row, 'StudyInstanceUID']
    dt_file =pydicom.dcmread( X[row])
    img = dt_file.pixel_array
    if(train_image_level.loc[row,'boxes']!= train_image_level.loc[row,'boxes']) == False:
        boxes = ast.literal_eval(train_image_level.loc[row,'boxes'])
        
        for box in boxes:
            p = matplotlib.patches.Rectangle((box['x'],box['y']), box['width'], box['height'],
                                           ec = 'r',fc = 'none', lw = 2.)
            axs[row].add_patch(p)
    axs[row].imshow(img,cmap = plt.cm.bone)
    axs[row].set_title(train_image_level.loc[row,'label'].split(' ')[0])
    axs[row].set_xticklabels([])
    axs[row].set_yticklabels([])
    

In [ ]:
import PIL
from PIL import Image, ImageDraw, ImageFont
import tensorflow as tf
import tensorflow_io as tfio
img1 = pydicom.dcmread(X[0])
b = tf.constant(img1.pixel_array)


## Create a function to preprocess the images
1. It should Take an image filepath as input i.e. X[0]
2. Using pydicom it should convert the read file into `TENSORS`
3. Normalise the image ( convert colour channel from 0-225 to 0-1 ).
4. Resize the image to be of the shape of (416,416)
5. Return the modified image.

* Had to convert the images in to RGB to convert them into acceptable Tensor


In [ ]:
# Define image size
IMAGE_SIZE = 416

# create a function for preprocessing the images

def process_image(path):
    """
    takes an image file and turn image into a tensor
    """
    img=pydicom.dcmread(path)
    img = img.pixel_array
    img = np.array(img)
    
    if len(img.shape)<3 :
        w, h = img.shape
        ret = np.empty((w, h, 3), dtype=np.uint8)
        ret[:, :, 2] =  ret[:, :, 1] =  ret[:, :, 0] =  img
        img=ret 
    # read in image file path and turn image into a tensor
    image = img
    image = tf.convert_to_tensor(image, np.float32)
     #convert the colour channel values from 0-255 to 0-1
    image = tf.image.convert_image_dtype(image,tf.float32)
     # Resize the image to our desired value (224,224)
    image = tf.image.resize(image, size=[IMAGE_SIZE, IMAGE_SIZE])
    
    
    return image

In [ ]:
a = process_image(X[0])

## Turning our data into batches

we do about 32 (This is the batch size) images at a time (you can manually adjust the batchsize if need to)

In order to use TensorFLow effectively, we need our data in the form of Tensor Tuples which look like this: (image , label)

In [ ]:
# Create a simple function to return a tuple( image , label)